C4. Créer une base de données dans le respect du RGPD en élaborant les modèles conceptuels et physiques des données à partir des données préparées et en programmant leur import afin de stocker le jeu de données du projet.

1. choix des colonnes à garder et envoyer dans la base de données 

In [39]:
import pandas as pd

import os
import numpy as np
import psycopg2

from dotenv import load_dotenv
from pathlib import Path
import re
import csv

In [40]:
df = pd.read_csv("../scrap_final/csv_clean/A&L.csv")
df.head(1)

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit,gout,Contenance,Taux PG/VG,...,Section_De_Qui,Section_Marque,Section_conseil,Section_taux_nicotine,Section_Conservation,info,brand,Description_cleaned,saveur,Description_cleaned_normalized
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,Ragnarok Ultimate A&L,<p>Le e-liquide Ragnarok par A&L Ultimate est ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53645-thi...,5.9,fruit,10ml,50/50,...,Comme tout e-liquide pour cigarette électroniq...,NaN,NaN,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le e-liquide Ragnarok par A&L Ultimate est un ...,A&L,Le e-liquide Ragnarok par A&L Ultimate est un ...,"cassis, lime, fruits rouges, parfait",liquide ragnarok ultimate produitpret emploipo...


In [41]:
df.shape

(1533, 28)

In [42]:
df.columns

Index(['lien_produit', 'nom_produit', 'Title', 'Description', 'Features',
       'img_produit', 'prix_produit', 'gout', 'Contenance', 'Taux PG/VG',
       'Origine', 'Frais / Mentholé', 'Surboosté en arômes', 'Saveur',
       'Section_Qu_est_ce_que', 'Section_Pourquoi', 'Section_Comment',
       'Section_A_Qui', 'Section_De_Qui', 'Section_Marque', 'Section_conseil',
       'Section_taux_nicotine', 'Section_Conservation', 'info', 'brand',
       'Description_cleaned', 'saveur', 'Description_cleaned_normalized'],
      dtype='object')

Vérifier les valeurs nulles dans chaque colonne

In [43]:
null_counts = df.isnull().sum()
print("Nombre de valeurs nulles par colonne :\n", null_counts)

Nombre de valeurs nulles par colonne :
 lien_produit                         0
nom_produit                          0
Title                                0
Description                          0
Features                             0
img_produit                          0
prix_produit                         6
gout                                 0
Contenance                           2
Taux PG/VG                          18
Origine                              9
Frais / Mentholé                     3
Surboosté en arômes                  5
Saveur                              31
Section_Qu_est_ce_que             1352
Section_Pourquoi                  1498
Section_Comment                    429
Section_A_Qui                     1513
Section_De_Qui                    1501
Section_Marque                      73
Section_conseil                    213
Section_taux_nicotine               66
Section_Conservation                55
info                                 0
brand                   

-------------------------------------------------------------------------------
table 

![alt text](../RGBD/img_table/table.png)

# conformité des brands 

In [45]:
# définir le dictionnaire de correspondance
brand_mapping = {
                'A&L': 'Arômes et Liquides (A&L)',
                'Space\xa0Fruit': 'Space Fruit'
}

# Appliquer le mapping à la colonne `brand`
df['brand'] = df['brand'].replace(brand_mapping)


In [46]:
df.brand.isnull().sum()

0

pour afficher que le nom du produit sans le brand a coté 
- degager le brand du  nom du produit
- degager des chiffres ml 

In [47]:
df.columns

Index(['lien_produit', 'nom_produit', 'Title', 'Description', 'Features',
       'img_produit', 'prix_produit', 'gout', 'Contenance', 'Taux PG/VG',
       'Origine', 'Frais / Mentholé', 'Surboosté en arômes', 'Saveur',
       'Section_Qu_est_ce_que', 'Section_Pourquoi', 'Section_Comment',
       'Section_A_Qui', 'Section_De_Qui', 'Section_Marque', 'Section_conseil',
       'Section_taux_nicotine', 'Section_Conservation', 'info', 'brand',
       'Description_cleaned', 'saveur', 'Description_cleaned_normalized'],
      dtype='object')

1. choix des colonnes à garder
lien_produit, Title, img_produit, prix_produit, Contenance, Taux PG/VG, Origine, Frais / Mentholé, Surboosté en arômes, Saveur, Section_Qu_est_ce_que, Section_Pourquoi, Section_Comment, Section_A_Qui, Section_De_Qui, Section_Marque, info	brand	Description_cleaned	saveur

In [49]:
df_table = df[["lien_produit", "Title", "img_produit", "prix_produit", "Contenance", "Taux PG/VG", "Origine", "Frais / Mentholé", "Surboosté en arômes", "saveur", "Description_cleaned", "Section_Marque", "brand", "gout"]]
df_table

,lien_produit,Title,img_produit,prix_produit,Contenance,Taux PG/VG,Origine,Frais / Mentholé,Surboosté en arômes,saveur,Description_cleaned,Section_Marque,brand,gout
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,https://assets.aromes-et-liquides.fr/53645-thi...,5.90,10ml,50/50,France,Oui,Non,"cassis, lime, fruits rouges, parfait",Le e-liquide Ragnarok par A&L Ultimate est un ...,NaN,Arômes et Liquides (A&L),fruit
1,https://www.aromes-et-liquides.fr/e-liquide-ul...,Oni Ultimate A&L,https://assets.aromes-et-liquides.fr/53646-thi...,5.90,10ml,50/50,France,Oui,Non,"citron vert, lime, parfait, citron",Le Oni Ultimate 10ml est un e-liquide pour cig...,NaN,Arômes et Liquides (A&L),fruit
2,https://www.aromes-et-liquides.fr/e-liquide-al...,A&L Fruits Rouges,https://assets.aromes-et-liquides.fr/53641-thi...,4.90,10ml,50/50,France,Non,Non,"baies, baie, fruits rouges, lime, baies rouges",Le Fruits Rouges est un e-liquide auxsaveurs d...,Arômes et Liquides (A&L) est un fabricant fran...,Arômes et Liquides (A&L),fruit
3,https://www.aromes-et-liquides.fr/e-liquide-he...,Red Light 200ml Hello Cloudy,https://assets.aromes-et-liquides.fr/45672-thi...,11.34,200ml,50/50,France,Oui,Oui,"lime, baie, baies, baies rouges","Le Red Light, unesaveur fraîche de baies rouge...",Hello Cloudy est un fabricant français de e-li...,Hello Cloudy,fruit
4,https://www.aromes-et-liquides.fr/e-liquide-t-...,Red Astaire T-Juice,https://assets.aromes-et-liquides.fr/55065-thi...,4.90,10ml,50/50,Royaume-Uni,Oui,Non,"fruits rouges, raisin, menthol, anis, lime, me...",Le Red Astaire est un e-liquide qui a rencontr...,NaN,T-Juice,fruit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,https://www.aromes-et-liquides.fr/e-liquide-al...,Blond Mentholé Sel de Nicotine Pur Distillat A...,https://assets.aromes-et-liquides.fr/55765-thi...,5.90,10ml,50/50,France,Non,Non,"tabac, menthol, anis, lime, menthe",Voici la recette du Blond Mentholé de chez Alf...,Le fabricant français s'est fait reconnaître p...,Alfaliquid,classique
1529,https://www.aromes-et-liquides.fr/e-liquide-81...,Nominoë 50ml 814,https://assets.aromes-et-liquides.fr/56383-thi...,21.90,50ml (fiole de 60ml),50/50,France,Non,Oui,"caramel, lime, biscuit",Le Nominoë est un e-liquide à booster au forma...,Le fabricant français 814 s'est spécialisé dan...,814,classique
1530,https://www.aromes-et-liquides.fr/e-liquide-th...,Dervish Fine Cuts Titanide The Fuu,https://assets.aromes-et-liquides.fr/55575-thi...,6.90,10ml,50/50,France,Non,Non,"anis, tabac",La marque The Fuu à travers sa collection Fine...,The Fuu est une marque française de liquides p...,The Fuu,classique
1531,https://www.aromes-et-liquides.fr/e-liquide-be...,Gold Digger Salt Ben Northon,https://assets.aromes-et-liquides.fr/56423-thi...,5.90,10ml,50/50,France,Non,Non,"anis, lime, tabac",La marque française Ben Northon vous propose u...,Le fabricant Ben Northon s'est spécialisé dans...,Ben Northon,classique


info brand 
* supprimer mes valeurs manquant de Section_Marque
* summprimer mes valeurs doublants de brand

In [50]:
df_brand = df[["Section_Marque", "brand"]]
df_brand = df_brand.dropna(subset=['Section_Marque'])
df_brand = df_brand.drop_duplicates(subset=['brand'])
df_brand

,Section_Marque,brand
2,Arômes et Liquides (A&L) est un fabricant fran...,Arômes et Liquides (A&L)
3,Hello Cloudy est un fabricant français de e-li...,Hello Cloudy
8,"C'est dans leur laboratoire français, et plus ...",PULP
10,Le fabricant français s'est fait reconnaître p...,Alfaliquid
15,Ce fabricant français élabore toutes ses recet...,Secret's Lab
...,...,...
1208,Liquidarom est une marque française de liquide...,Liquidarom
1356,Fabricant reconnu pour ses cartouches pré-remp...,Elfbar
1430,Le fabricant français Terroir et Vapeur centra...,Terroir et Vapeur
1457,Le fabricant Ben Northon s'est spécialisé dans...,Ben Northon


affecter la valeur de la colonne Section_Marque de df par la valeur correspondante dans df_brand lorsque la valeur dans df['Section_Marque'] est NaN et que df['brand'] correspond à df_brand['brand']

In [51]:
# Fusionner df avec df_brand en fonction de la colonne 'brand' pour obtenir la colonne Section_Marque de df_brand
df_table = pd.merge(df_table, df_brand[['brand', 'Section_Marque']], on='brand', how='left', suffixes=('', '_brand'))

In [52]:
# Supprimer la colonne 'Section_Marque_brand' après la mise à jour
df_table.drop(columns=['Section_Marque'], inplace=True)

retirer les valeurs de la colonne Title qui correspondent aux valeurs de la colonne brand
retirer les ml

In [53]:

df['Title'] = df.apply(lambda row: row['Title'].replace(row['brand'], '') if row['brand'].lower() in row['Title'].lower() else row['Title'], axis=1)
df['Title'] = df['Title'].apply(lambda x: re.sub(r'\d+ml|\d+', '', x).replace('ml', '').strip())

In [54]:
df_table.head(1)

,lien_produit,Title,img_produit,prix_produit,Contenance,Taux PG/VG,Origine,Frais / Mentholé,Surboosté en arômes,saveur,Description_cleaned,brand,gout,Section_Marque_brand
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,https://assets.aromes-et-liquides.fr/53645-thi...,5.9,10ml,50/50,France,Oui,Non,"cassis, lime, fruits rouges, parfait",Le e-liquide Ragnarok par A&L Ultimate est un ...,Arômes et Liquides (A&L),fruit,Arômes et Liquides (A&L) est un fabricant fran...


check les double Title et lien_produit

In [55]:
df_table["Title"].nunique()

1533

In [56]:
df_table["lien_produit"].nunique()

1533

création de id_brand 

In [57]:
# Créer un dictionnaire pour mapper chaque marque unique à un ID unique
id_produit = {lien_produit: idx for idx, lien_produit in enumerate(df_table['lien_produit'].unique(), start=1)}
# Ajouter une colonne `id_brand` au df_brand
df_table['id_produit'] = df_table['lien_produit'].map(id_produit)

changer le nom de colonnes

In [58]:
df_table.columns

Index(['lien_produit', 'Title', 'img_produit', 'prix_produit', 'Contenance',
       'Taux PG/VG', 'Origine', 'Frais / Mentholé', 'Surboosté en arômes',
       'saveur', 'Description_cleaned', 'brand', 'gout',
       'Section_Marque_brand', 'id_produit'],
      dtype='object')

In [59]:
# Renommer les colonnes
df_table.rename(columns={'lien_produit': 'url', 'Title': 'nom_produit', 'Contenance': 'contenance', 'Taux PG/VG' :'pg_vg', 'Origine': 'origine', 'Frais / Mentholé': 'frais', 'Surboosté en arômes': 'surbooste', 'Description_cleaned': 'description', 'Section_Marque_brand': 'info_brand'}, inplace=True)

In [60]:
df_table.columns

Index(['url', 'nom_produit', 'img_produit', 'prix_produit', 'contenance',
       'pg_vg', 'origine', 'frais', 'surbooste', 'saveur', 'description',
       'brand', 'gout', 'info_brand', 'id_produit'],
      dtype='object')

In [61]:
# Remplacer les sauts de ligne par un espace
df_table['description'] = df_table['description'].str.replace('\n', ' ', regex=True)
df_table['info_brand'] = df_table['info_brand'].str.replace('\n', ' ', regex=True)

In [62]:
df_table.isnull().sum()

url              0
nom_produit      0
img_produit      0
prix_produit     6
contenance       2
pg_vg           18
origine          9
frais            3
surbooste        5
saveur           2
description      0
brand            0
gout             0
info_brand      10
id_produit       0
dtype: int64

In [63]:
df_table[df_table['info_brand'].isna()]

,url,nom_produit,img_produit,prix_produit,contenance,pg_vg,origine,frais,surbooste,saveur,description,brand,gout,info_brand,id_produit
101,https://www.aromes-et-liquides.fr/e-liquide-fr...,Grapefruit Lime Ice 100ml Frozen Freaks,https://assets.aromes-et-liquides.fr/36936-thi...,10.95,100ml (fiole de 120ml),30/70,Royaume-Uni,Oui,Oui,"lime, citron, pamplemousse, citron vert, agrumes","Besoin d'un bon coup de fouet fruité, frais et...",Frozen Freaks,fruit,NaN,102
588,https://www.aromes-et-liquides.fr/e-liquide-pu...,Granité de Melon 60ml Pulp,https://assets.aromes-et-liquides.fr/54046-thi...,22.90,50ml (fiole de 60ml),70/30,France,Oui,Oui,"lime, melon",Découvrez une recette au goût frais et fruité ...,Pulp,fruit,NaN,589
606,https://www.aromes-et-liquides.fr/e-liquide-pu...,Pomme Chicha 60ml Pulp,https://assets.aromes-et-liquides.fr/54049-thi...,22.90,50ml (fiole de 60ml),30/70,France,Non,Oui,"lime, réglisse, pomme",Découvrez unesaveur de pomme façon chicha et a...,Pulp,fruit,NaN,607
651,https://www.aromes-et-liquides.fr/e-liquide-th...,Pêche Mango 50ml Puff Juice,https://assets.aromes-et-liquides.fr/52678-thi...,19.90,50ml (fiole de 75ml),40/60,France,Non,Oui,"pêche, lime, vin, mangue","Quand les tropiques rencontrent le verger, cel...",Puff Juice,fruit,NaN,652
676,https://www.aromes-et-liquides.fr/e-liquide-th...,Fruits Rouges 50ml Puff Juice,https://assets.aromes-et-liquides.fr/52682-thi...,19.90,50ml (fiole de 75ml),40/60,France,Non,Oui,"baies, fruits des bois, fruits rouges, baie, lime","Il est des saveurs que l'on ne présente plus, ...",Puff Juice,fruit,NaN,677
700,https://www.aromes-et-liquides.fr/e-liquide-th...,Raisin Givré 50ml Puff Juice,https://assets.aromes-et-liquides.fr/52680-thi...,19.90,50ml (fiole de 75ml),40/60,France,Oui,Oui,"lime, glace, raisin",Les vendanges ont été fructueuses chez The Fuu...,Puff Juice,fruit,NaN,701
703,https://www.aromes-et-liquides.fr/e-liquide-pu...,Cassis Exquis 60ml Pulp,https://assets.aromes-et-liquides.fr/54045-thi...,22.90,50ml (fiole de 60ml),70/30,France,Non,Oui,"cassis, lime",Le fabricant français vous propose de vapoter ...,Pulp,fruit,NaN,704
1200,https://www.aromes-et-liquides.fr/e-liquide-th...,Charlotte aux Fraises 50ml Puff Juice,https://assets.aromes-et-liquides.fr/52679-thi...,19.90,50ml (fiole de 75ml),40/60,France,Non,Oui,"fraise, lime, biscuit",The Fuu vous propose un véritable dessert à va...,Puff Juice,dessert,NaN,1201
1343,https://www.aromes-et-liquides.fr/e-liquide-pu...,Le Thé à la Menthe 60ml Pulp,https://assets.aromes-et-liquides.fr/54047-thi...,22.90,50ml (fiole de 60ml),70/30,France,Oui,Oui,"thé, lime, menthol, menthe",Pulp vous propose de vapoter une saveur de boi...,Pulp,frais,NaN,1344
1347,https://www.aromes-et-liquides.fr/e-liquide-th...,Tropical Icy Mix 50ml Puff Juice,https://assets.aromes-et-liquides.fr/52681-thi...,19.90,50ml (fiole de 75ml),40/60,France,Oui,Oui,"ananas, cocktail, mangue, banane, glace, lime",Le fabricant français vous propose un voyage g...,Puff Juice,frais,NaN,1348


info_brand null --> Aucune information sur la marque

In [65]:
df_table['info_brand'] = df_table['info_brand'].fillna("Aucune information sur la marque")


In [74]:
df_table = df_table.dropna(subset=['saveur'])

In [78]:
df_table[['contenance','pg_vg', 'origine', 'frais', 'surbooste']] = df_table[['contenance','pg_vg', 'origine', 'frais', 'surbooste']].fillna('Inconnu')


In [79]:
df_table.isnull().sum()

url             0
nom_produit     0
img_produit     0
prix_produit    6
contenance      0
pg_vg           0
origine         0
frais           0
surbooste       0
saveur          0
description     0
brand           0
gout            0
info_brand      0
id_produit      0
dtype: int64

In [82]:
df_table.columns

Index(['url', 'nom_produit', 'img_produit', 'prix_produit', 'contenance',
       'pg_vg', 'origine', 'frais', 'surbooste', 'saveur', 'description',
       'brand', 'gout', 'info_brand', 'id_produit'],
      dtype='object')

In [81]:
import csv  # Assurez-vous d'importer le module csv

df_table.to_csv('../RGBD/table_produits/produits.csv', header=df_table.columns, index=False, encoding='utf-8', quotechar='"', quoting=csv.QUOTE_MINIMAL)
